# Введение в PyTorch

<img src="./images/pytorch_logo.png" width=240>

**На этом уроке мы рассмотрим как при помощи PyTorch:**
- Работать с данными 
- Создавать свои модели нейросетей (на примере полносвязных и сверточных сетей)
- Обучать созданные модели
- Сохранять и загружать обученные модели (checkpoints)
- Делать предсказания (инференс)
- Ускорить инференс при помощи компиляции моделей (PyTorch 2.0)
- Transfer Lerning из одного датасета в другой

В качестве датасетов будем использовать MNIST и CIFAR10. 

**Для начала импортируем необходимые библиотеки:**

In [ ]:
import numpy as np
import torch
torch.__version__

In [ ]:
# Вспомогательная функция для визуализации данных:
def show(img, title=''):
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap="gray")
    plt.show()

## 1 Работа с данными

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

**Загрузка датасета**

In [ ]:
# Скачиваем данные для обучения:
training_data = datasets.MNIST(
    root="./dataset",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

# Скачиваем данные для теста:
test_data = datasets.MNIST(
    root="./dataset",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

**Визуализация данных**

In [ ]:
import matplotlib.pyplot as plt

# Опишем название классов:
labels_map = ['0','1','2','3','4','5','6','7','8','9']
 
# Визуализируем рандомные сэмплы из MNIST:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray") # squeeze() - удаляем все измерения размером 1
plt.show()

**Итерирование через данные**

In [ ]:
batch_size = 64

# Создаем data loaders:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")

## 2 Создание модели
<img src="images/nn_picture.png">

#### Слои
[**Flatten**](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) -  трансформирует 2D изображение (28x28 пикселей) в вектор (784 пикселей).

[**Linear**](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) -  применяет линейную трансформацию к входным данным используя веса модели и смещение (bias):
<img src="images/linear.png">

[**ReLU**](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) -  функция активации, добавляет нейлинейность в модель, что позволяет модели аппроксимировать нейлинейные функции:
<img src="images/relu.png">

[**Sequential**](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) -  упорядоченный контейнер модулей нейронной сети, предназначенный для быстрого создания сетей с последовательной архитектурой. Данные будут проходить слои модели в том порядке, в котором они описаны. 

[**Softmax**](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) -  это обобщение логистической функции на несколько измерений, используется для нормализации выходных данных сети (в диапазон значений [0,1], которые в сумме дают 1):
<img src="images/softmax.png">

In [ ]:
# Выбираем девайс (cpu или gpu), на котором будут происходит вычисления:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Определяем кастомную модель (Sequential):
class NeuralNetworkSeq(nn.Module):
    def __init__(self, input_size=(28,28,1), output_size=10):
        super(NeuralNetworkSeq, self).__init__()
        self.flatten = nn.Flatten()
        self.output = nn.Linear(512, output_size)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size[0]*input_size[1]*input_size[2], 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            self.output
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model_seq = NeuralNetworkSeq().to(device)
print(model_seq)

In [ ]:
# Определяем кастомную модель:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=(28,28,1), output_size=10):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size[0]*input_size[1]*input_size[2], 512)
        self.fc2 = nn.Linear(512, 512)
        self.relu = nn.ReLU(inplace=True)
        self.output = nn.Linear(512, output_size)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        logits = self.output(x)
        
        return logits


model = NeuralNetwork().to(device)
print(model)

In [ ]:
# Мы можем итерироваться через параметры (веса) нашей модели:
for name, param in model.named_parameters():
    print(f"Имя: {name} | Размер: {param.size()} | Значения : {param[:2]} \n")

In [ ]:
# Пропустим рандомное изображение из MNIST через нашу еще необученную модель:
sample_idx = torch.randint(len(training_data), size=(1,)).item()
X, Y = training_data[sample_idx]
X = X.to(device)

# Forward pass:
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
Y_pred = pred_prob.argmax(1)

print('Ненормализованные вероятности:\n', logits.cpu().detach())
print('Нормализованные вероятности:\n', pred_prob.cpu().detach())
text = f"Prediction: {Y_pred.item()} / Ground Truth: {Y}"
show(X.cpu().squeeze(), title=text)

## 3 Обучение (оптимизация параметров модели)

In [ ]:
# Функция для обучения модели:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Делаем предсказания:
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
# Функция для тестирования обученной модели:
def test(dataloader, model, loss_fn, verbose=True, iterations=None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if iterations is not None and i >= iterations:
                break
            
    test_loss /= num_batches
    correct /= size
    if verbose:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")  

### Функции потерь / Loss Functions

[**MSELoss**](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) -  Mean Square Error / Squared L2 (среднеквадратичная ошибка / квадратичная L2 норма:
<img src="images/mse.png">

[**CrossEntropyLoss**](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) -  Перекрестная энтропия, комбинирует Softmax и Negative Log Likelihood:
<img src="images/crossentropy.png">

### Оптимайзеры / Optimizers
[**SGD**](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html?highlight=sgd#torch.optim.SGD) -  Стохастический градиентный спуск, принимает аргументы:
- параметры модели, 
- learning rate (скорость обучения),
- momentum (импульс), 
- weight_decay (распад весов, помогает регуляризировать модель, тем самым предотвращая переобучения).

[**Adam**](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html?highlight=adam#torch.optim.Adam) -  Оптимизационный алгоритм Адама:
- параметры модели, 
- learning rate (скорость обучения),
- betas, 
- weight_decay.

In [ ]:
# Инициализируем модель и data loaders:
model = NeuralNetwork().to(device)
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Задаем гипперпараметры:
epochs = 5
batch_size = 64
lr = 1e-3

# Выбираем функцию потерь и оптимайзер:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Обучаем 5 эпох (эпоха - один проход по всем данным):
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## 4 Сохранение и загрузка обученной модели

In [ ]:
# Сохраняем checkpoint:
checkpoint_path = './checkpoints/mnist_checkpoint.pth'
torch.save(model.state_dict(), checkpoint_path)
print("Saved PyTorch Model State to {}".format(checkpoint_path))

In [ ]:
# Создаем модель (объект класса NeuralNetwork) и загружаем параметры из checkpoint:
model = NeuralNetwork()
model.load_state_dict(torch.load(checkpoint_path))
print (model.state_dict())

In [ ]:
# Протестируем загруженную модель (чтобы убедиться что все правильно загрузилось):
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
test(test_dataloader, model, loss_fn)

## 5 Делаем предсказания (inference)

In [ ]:
# Опишем название классов:
classes = ['0','1','2','3','4','5','6','7','8','9']

sample_idx = torch.randint(len(test_data), size=(1,)).item()
x, y = test_data[sample_idx][0], test_data[sample_idx][1]
x = x.to(device)

model.eval()
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    text = f"Prediction: {predicted} / Ground Truth: {actual}"
    show(x.cpu().squeeze(), title=text)

## 6 Обучаем сверточную сеть
LeNet архитектура:
<img src="images/cnn.jpg">

In [ ]:
import torch.nn.functional as F     # imports activation functions

class CNN(nn.Module):
    def __init__(self, out_size):
        super(CNN, self).__init__()
        # Describe layers:
        self.conv1 = nn.Conv2d(1, 32, 3, 1)     # 1: in channels, 32: out channels, 3: kernel size, 1: stride
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)         # 9216: in channels, 128: out channels
        self.fc2 = nn.Linear(128, out_size)

    # Forward propagation:
    def forward(self, x):
        '''
        x reprenets our input data
        '''
        # Pass data through conv's layers:
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        # Run max pooling over x:
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        
        # Flatten x with start_dim=1
        x = torch.flatten(x, 1)
        
        # Pass data through FC's layers:
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        return x
    
cnn_model = CNN(10).to(device)
print(cnn_model)

In [ ]:
# Инициализируем модель и data loaders:
cnn_model = CNN(10).to(device)
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Задаем гипперпараметры:
epochs = 5
batch_size = 64
lr = 1e-3

# Выбираем функцию потерь и оптимайзер:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(cnn_model.parameters(), lr=lr, momentum=0.9)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=lr, betas=(0.9, 0.999))

# Обучаем 5 эпох:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, cnn_model, loss_fn, optimizer)
    test(test_dataloader, cnn_model, loss_fn)
print("Done!")

In [ ]:
# Протестируем обученную модель:
cnn_model = cnn_model.to(device)
loss_fn = nn.CrossEntropyLoss()
test(test_dataloader, cnn_model, loss_fn)

In [ ]:
# Опишем название классов:
classes = ['0','1','2','3','4','5','6','7','8','9']

# Делаем предсказание:
sample_idx = 444
x, y = test_data[sample_idx][0], test_data[sample_idx][1]
x = x.to(device)

cnn_model.eval()
with torch.no_grad():
    pred = cnn_model(x.unsqueeze(0))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    text = f"Prediction: {predicted} / Ground Truth: {actual}"
    show(x.cpu().squeeze(), title=text)

# Ускорерние инференеса в PyTorch 2.0

`torch.compile(model=None, *, fullgraph=False, dynamic=False, backend='inductor', mode=None, options=None, disable=False)`

Optimizes given model/function using TorchDynamo and specified backend.

Parameters:
- model (Callable) – Module/function to optimize
- fullgraph (bool) – Whether it is ok to break model into several subgraphs
- dynamic (bool) – Use dynamic shape tracing
- backend (str or Callable) – backend to be used
- mode (str) – Can be either “default”, “reduce-overhead” or “max-autotune”
- options (dict) – A dictionary of options to pass to the backend.
- disable (bool) – Turn torch.compile() into a no-op for testing

## Компиляция кастомной модели

In [ ]:
import time

# Модель и лосс функция:
cnn_model = cnn_model.to(device)
loss_fn = nn.CrossEntropyLoss()


# Замерим время инференса модели без компиляции:
start = time.time()
for i in range(10):
    test(test_dataloader, cnn_model, loss_fn, verbose=False)
print('W/O compile:', time.time() - start)


# Warm-up для компиляции:
compiled_cnn_model_ro = torch.compile(cnn_model, mode="reduce-overhead")
test(test_dataloader, compiled_cnn_model_ro, loss_fn, verbose=False)


# Компилируем с defualt параметрами: 
# - быстрое время компиляции
# - не использует дополнительную память для компиляции
compiled_cnn_model = torch.compile(cnn_model)
start = time.time()
for i in range(10):
    test(test_dataloader, compiled_cnn_model, loss_fn, verbose=False)
print('default:', time.time() - start)


# Компиляция reduce-overhead:
# - хорошо ускоряет небольшие модели
# - снижает накладные расходы фреймворка
# - использует дополнительную память для компиляци
compiled_cnn_model_ro = torch.compile(cnn_model, mode="reduce-overhead")
start = time.time()
for i in range(10):
    test(test_dataloader, compiled_cnn_model_ro, loss_fn, verbose=False)
print('reduce-overhead:', time.time() - start)


# Компиляция max-autotune: 
# - выдает максимально быстрый код,
# - занимает значительное время для компиляции
compiled_cnn_model_ma = torch.compile(cnn_model, mode="max-autotune")
start = time.time()
for i in range(10):
    test(test_dataloader, compiled_cnn_model_ma, loss_fn, verbose=False)
print('max-autotune:', time.time() - start)

## Компиляция моделей torchvision

In [ ]:
import time
import torch
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size=32


# Скачиваем CIFAR датасет и приводим его к размеру 224x224 для тестирования torchvision моделей:
test_data_cifar10_224 = datasets.CIFAR10(
    root="./dataset_cifar",
    train=False,
    download=True,
    transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])
)

test_dataloader_cifar10_224 = DataLoader(test_data_cifar10_224, batch_size=batch_size)


# Функция для сравнения скомплириованных и оригинальных моделей:
#@torch._dynamo.optimize('inductor')
def compare(model, iters=1, test_batches=None):
    '''
    iters:        число запусков теста
    test_batches: число батчей в тесте
    '''
    # Без компиляции:
    start = time.time()
    for i in range(iters):
        test(test_dataloader_cifar10_224, vision_model, nn.CrossEntropyLoss(), False, test_batches)
    wo_elapsed = time.time() - start
    print('W/O compile:', wo_elapsed)
    
    
    # Warm-up compilation:
#     torch._dynamo.reset()
    compiled_vision_model = torch.compile(vision_model, mode="reduce-overhead")
    test(test_dataloader_cifar10_224, compiled_vision_model, nn.CrossEntropyLoss(), False, test_batches)


    # С компиляцией default:
    compiled_vision_model = torch.compile(vision_model)
    start = time.time()
    for i in range(iters):
        test(test_dataloader_cifar10_224, compiled_vision_model, nn.CrossEntropyLoss(), False, test_batches)
    elapsed = time.time() - start
    print(f'default: {elapsed}, speedup {wo_elapsed/elapsed}')


    # С компиляцией reduce-overhead:
    compiled_vision_model = torch.compile(vision_model, mode="reduce-overhead")
    start = time.time()
    for i in range(iters):
        test(test_dataloader_cifar10_224, compiled_vision_model, nn.CrossEntropyLoss(), False, test_batches)
    elapsed = time.time() - start
    print(f'reduce-overhead: {elapsed}, speedup {wo_elapsed/elapsed}')


    # С компиляцией max-autotune:
    compiled_vision_model = torch.compile(vision_model, mode="max-autotune")
    start = time.time()
    for i in range(iters):
        test(test_dataloader_cifar10_224, compiled_vision_model, nn.CrossEntropyLoss(), False, test_batches)
    elapsed = time.time() - start
    print(f'max-autotune: {elapsed}, speedup {wo_elapsed/elapsed}')

In [ ]:
iters, test_batches = 3, 10

# Выбираем torchvision модель:
vision_model = torchvision.models.resnet50().to(device)
print('\n### resnet50 ###')
compare(vision_model, iters, test_batches)


vision_model = torchvision.models.mobilenet_v3_small().to(device)
print('\n### mobilenet_v3_small ###')
compare(vision_model, iters, test_batches)


vision_model = torchvision.models.mobilenet_v3_large().to(device)
print('\n### mobilenet_v3_large ###')
compare(vision_model, iters, test_batches)


vision_model = torchvision.models.efficientnet_v2_s().to(device)
print('\n### efficientnet_v2_s ###')
compare(vision_model, iters, test_batches)


vision_model = torchvision.models.efficientnet_v2_s().to(device)
print('\n### efficientnet_v2_l ###')
compare(vision_model, iters, test_batches)

In [ ]:
#from torch import dynamo
#print (torch._dynamo.list_backends())

# Swin трансформер:
vision_model = torchvision.models.swin_s().to(device)
print('\n### swin_s ###')
compare(vision_model, 2, 10)

## 7 Обучаем полносвязную сеть на CIFAR10 (rgb, 32x32, 10 классов)

In [ ]:
# Скачиваем данные для обучения:
training_data_cifar10 = datasets.CIFAR10(
    root="./dataset_cifar",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

# Скачиваем данные для теста:
test_data_cifar10 = datasets.CIFAR10(
    root="./dataset_cifar",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

**Визуализация данных**

In [ ]:
import matplotlib.pyplot as plt

# Опишем название классов:
labels_map = ['airplanes','cars','birds','cats','deer','dogs','frogs','horses','ships','trucks']
 
# Визуализируем рандомные сэмплы из MNIST:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data_cifar10), size=(1,)).item()
    img, label = training_data_cifar10[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.permute(1, 2, 0))    # переставляем каналы [C,H,W] -> [H,W,C]
plt.show()

**Обучаем модель на CIFAR10**

In [ ]:
batch_size = 64

# Инициализируем модель и data loaders:
model = NeuralNetwork(input_size=(32,32,3)).to(device)
train_dataloader_cifar10 = DataLoader(training_data_cifar10, batch_size=batch_size)
test_dataloader_cifar10 = DataLoader(test_data_cifar10, batch_size=batch_size)

# Задаем гипперпараметры:
epochs = 10
batch_size = 64
lr = 1e-3

# Выбираем функцию потерь и оптимайзер:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Обучаем 5 эпох (эпоха - один проход по всем данным):
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader_cifar10, model, loss_fn, optimizer)
    test(test_dataloader_cifar10, model, loss_fn)
print("Done!")

## 8 Обучаем полносвязную сеть на CIFAR10 (grayscale, 28x28, 10 классов)

In [ ]:
# Скачиваем данные для обучения:
training_data_cifar10 = datasets.CIFAR10(
    root="./dataset_cifar",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28,28)),
        transforms.ToTensor()])
)

# Скачиваем данные для теста:
test_data_cifar10 = datasets.CIFAR10(
    root="./dataset_cifar",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.Grayscale(num_output_channels=1), 
        transforms.Resize((28,28)),
        transforms.ToTensor()])
)

In [ ]:
import matplotlib.pyplot as plt

# Опишем название классов:
labels_map = ['airplanes','cars','birds','cats','deer','dogs','frogs','horses','ships','trucks']
 
# Визуализируем рандомные сэмплы из MNIST:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data_cifar10), size=(1,)).item()
    img, label = training_data_cifar10[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray") # squeeze() - удаляем все измерения размером 1
plt.show()

**Обучаем модель на CIFAR10 (grayscale, 28x28)**

In [ ]:
# Инициализируем модель и data loaders:
model = NeuralNetwork(input_size=(28,28,1)).to(device)
train_dataloader_cifar10 = DataLoader(training_data_cifar10, batch_size=batch_size)
test_dataloader_cifar10 = DataLoader(test_data_cifar10, batch_size=batch_size)

# Задаем гипперпараметры:
epochs = 10
batch_size = 64
lr = 1e-3

# Выбираем функцию потерь и оптимайзер:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Обучаем 5 эпох (эпоха - один проход по всем данным):
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader_cifar10, model, loss_fn, optimizer)
    test(test_dataloader_cifar10, model, loss_fn)
print("Done!")

## 9 Transfer Learning MNIST -> CIFAR10

In [ ]:
# Загружаем веса MNIST из checkpoint:
checkpoint_path = './checkpoints/mnist_checkpoint.pth'
model = NeuralNetwork()
model.load_state_dict(torch.load(checkpoint_path))

# Протестируем загруженную модель на MNIST:
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
test(test_dataloader, model, loss_fn)

**Заморозим веса всех слоев кроме последнего**

In [ ]:
# Отключаем градиент (кроме output слоя):
for param in model.parameters(): # Проходим по параметрам модели (каждый параметр - это каждый слой)
    param.requires_grad = False
    
# Пересоздаем выходной слой на 10 нейронов:
mnist_output = model.output                       # Сохраняем выходной слой MNIST
in_features = model.output.in_features
model.output = torch.nn.Linear(in_features, 10, bias=True)
model.output = model.output.cuda()                # Перенесем на cuda

# Проверяем:
for name, param in model.named_parameters():
    print (name, param.shape, param.requires_grad)

**Протестируем модель, обученную на MNIST, на CIFAR**

In [ ]:
# Инициализируем data loaders:
train_dataloader_cifar10 = DataLoader(training_data_cifar10, batch_size=batch_size)
test_dataloader_cifar10 = DataLoader(test_data_cifar10, batch_size=batch_size)

# Протестируем модель, обученную на MNIST, на CIFAR:
test(test_dataloader_cifar10, model, loss_fn)

**Дообучим последний слой модели на CIFAR**

In [ ]:
# Задаем гипперпараметры:
epochs = 10
batch_size = 64
lr = 1e-3

# Выбираем функцию потерь и оптимайзер:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Обучаем 5 эпох (эпоха - один проход по всем данным):
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader_cifar10, model, loss_fn, optimizer)
    test(test_dataloader_cifar10, model, loss_fn)
print("Done!")

In [ ]:
# Восстановим выходной слой MNIST:
model.output = mnist_output

# Протестируем модель еще раз на MNIST:
loss_fn = nn.CrossEntropyLoss()
test(test_dataloader, model, loss_fn)

## Теперь вы знаете как создать и обучить свою нейронную сеть!

In [ ]:
# Сохраняем checkpoint:
checkpoint_path = 'Checkpoints/cnn_mnist_checkpoint.pth'
torch.save(cnn_model.state_dict(), checkpoint_path)

In [ ]:
# Загружаем checkpoint:
checkpoint_path = 'Checkpoints/cnn_mnist_checkpoint.pth'
cnn_model = CNN(10)
cnn_model.load_state_dict(torch.load(checkpoint_path))

In [ ]:
import coremltools as ct

# Сохраняем модель как TorchScript:
cnn_model.eval()
example_input = torch.rand(1, 1, 28, 28) 
traced_cnn_model = torch.jit.trace(cnn_model, example_input)
out = traced_cnn_model(example_input)

# Convert to Core ML using the Unified Conversion API:
coreml_model = ct.convert(
    traced_cnn_model,
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model:
coreml_model.save("Checkpoints/cnn_mnist.mlmodel")

In [ ]:
import time
import torch
import torchvision
from torch import _dynamo


print (torch._dynamo.list_backends())

@torch._dynamo.optimize('inductor')
def fun(x):
    x = torch.add(x, 1)
    x = torch.sub(x, 1)
    x = torch.add(x, 1)
    return x

tensor = torch.tensor((500,500,700))
fun(tensor)
start = time.time()
for i in range(1000):
    fun(tensor)
print(time.time() - start)